# Agentic AI Assistant - ShopUNow

This notebook builds an AI assistant for routing queries, detecting sentiment, and answering using RAG.  
It also includes a human-escalation form as the stretch goal.  
We will use Groq for all LLM calls.


In [ ]:
!pip install groq

In [ ]:
from groq import Groq
import os
from pydantic import BaseModel
from typing import Optional, Literal

In [ ]:
# Directly set your Groq API key here
groq_api_key = "xxx_xxx_xxxx"

if not groq_api_key:
    raise ValueError("Groq API key is not set.")

from groq import Groq
client = Groq(api_key=groq_api_key)

print("Groq client initialized successfully.")

Groq client initialized successfully.


In [ ]:
# Define departments exactly as provided

departments = {
    "human_resources": "Internal",
    "it_helpdesk": "Internal",
    "orders_shipping": "External",
    "returns_refunds": "External"
}

departments

{'human_resources': 'Internal',
 'it_helpdesk': 'Internal',
 'orders_shipping': 'External',
 'returns_refunds': 'External'}

In [ ]:
# Empty QA dataset structure for all departments
qa_data = {
    "human_resources": [],
    "it_helpdesk": [],
    "orders_shipping": [],
    "returns_refunds": []
}

qa_data

{'human_resources': [],
 'it_helpdesk': [],
 'orders_shipping': [],
 'returns_refunds': []}

### Human Resources QA Data
This section contains 15 QA pairs for the Human Resources department.

In [ ]:
# Add 15 QA pairs for Human Resources

qa_data["human_resources"] = [
    {
        "question": "What are the official working hours at ShopUNow?",
        "answer": "Standard working hours are 9 AM to 6 PM, Monday through Friday."
    },
    {
        "question": "How do I request paid time off?",
        "answer": "Submit a PTO request through the HR Portal under 'Leave Management' at least 3 business days in advance."
    },
    {
        "question": "How can I update my home address in company records?",
        "answer": "Log into the HR Portal and update your address under 'Personal Information'. Changes take effect within 48 hours."
    },
    {
        "question": "Who should I contact if my salary appears incorrect?",
        "answer": "Email hr-payroll@shopunow.com or raise a payroll ticket in the HR Portal."
    },
    {
        "question": "Does ShopUNow provide health insurance for employees?",
        "answer": "Yes, ShopUNow provides health, dental, and vision insurance starting on your 30th day of employment."
    },
    {
        "question": "How do I report workplace harassment?",
        "answer": "Submit a confidential complaint using the 'HR Complaints' section in the HR Portal or email hr-compliance@shopunow.com."
    },
    {
        "question": "When do performance reviews happen?",
        "answer": "Performance reviews occur annually in December, with mid-year check-ins in June."
    },
    {
        "question": "What is the dress code at ShopUNow?",
        "answer": "Business casual from Monday to Thursday and casual wear on Fridays."
    },
    {
        "question": "How do I apply for maternity leave?",
        "answer": "Submit a maternity leave request through the HR Portal under 'Leave Requests' at least 4 weeks prior to the expected date."
    },
    {
        "question": "Does ShopUNow allow remote work?",
        "answer": "Remote work is allowed for eligible roles with manager approval. Check the Remote Work Policy in the HR Portal."
    },
    {
        "question": "How can I access my previous salary slips?",
        "answer": "Salary slips can be downloaded anytime from the HR Portal under 'Payroll Documents'."
    },
    {
        "question": "What is the probation period for new employees?",
        "answer": "The probation period is 90 days from the date of joining."
    },
    {
        "question": "How do I refer someone for a job at ShopUNow?",
        "answer": "Use the Employee Referral section on the Careers Portal and submit the candidate's resume."
    },
    {
        "question": "Where can I find company holidays for the year?",
        "answer": "The holiday calendar is available in the HR Portal under 'Company Policies'."
    },
    {
        "question": "How do I change my tax withholding information?",
        "answer": "Update your tax details in the HR Portal under 'Payroll Settings'. Changes reflect in the next pay cycle."
    }
]

len(qa_data["human_resources"])


15

### IT Helpdesk QA Data
This section contains 15 QA pairs for the IT Helpdesk department.

In [ ]:
# Add 15 QA pairs for IT Helpdesk

qa_data["it_helpdesk"] = [
    {
        "question": "How do I reset my ShopUNow email password?",
        "answer": "Go to the IT Self-Service Portal and select 'Reset Password'. Follow the verification steps."
    },
    {
        "question": "My laptop is running slow. What should I do?",
        "answer": "Restart the device, clear temporary files, and ensure all updates are installed. If the issue continues, raise an IT ticket."
    },
    {
        "question": "How do I request a new laptop or equipment?",
        "answer": "Submit a hardware request in the IT Self-Service Portal under 'Equipment Requests'."
    },
    {
        "question": "Microsoft Teams is not opening. How can I fix it?",
        "answer": "Restart Teams, clear its cache, or reinstall the app. Instructions are available in the IT Portal."
    },
    {
        "question": "What should I do if my VPN is not connecting?",
        "answer": "Check your internet connection, restart the VPN client, and verify your credentials. If still failing, open an IT ticket."
    },
    {
        "question": "How do I access the company Wi-Fi on a new device?",
        "answer": "Use your employee credentials to connect to 'ShopUNow-Secure' following the steps in the IT Portal."
    },
    {
        "question": "Who should I contact for issues with my desktop setup?",
        "answer": "Raise a service ticket under 'Desktop Support' in the IT Portal."
    },
    {
        "question": "How can I update required software on my workstation?",
        "answer": "Open the Company Software Center and install required updates listed under 'Pending Installations'."
    },
    {
        "question": "I am receiving too many spam emails. What should I do?",
        "answer": "Report spam using the 'Report Junk' button in Outlook. IT will automatically adjust filters."
    },
    {
        "question": "How do I get access to internal applications?",
        "answer": "Submit an 'Access Request' in the IT Portal. Manager approval is required."
    },
    {
        "question": "My system is overheating. What is the fix?",
        "answer": "Check ventilation, close heavy applications, and restart the device. If overheating continues, schedule a hardware check."
    },
    {
        "question": "How do I set up two external monitors?",
        "answer": "Follow the dual-monitor setup guide in the IT Portal or raise a ticket for on-site support."
    },
    {
        "question": "Can I install personal software on my company laptop?",
        "answer": "No, personal software installations are not allowed unless approved by IT Security."
    },
    {
        "question": "What should I do if I clicked a suspicious link?",
        "answer": "Immediately disconnect from the network and contact IT Security through the high-priority alert hotline."
    },
    {
        "question": "How do I unlock my account after too many failed login attempts?",
        "answer": "Use the Self-Service Unlock feature in the IT Portal or contact IT Support to manually unlock your account."
    }
]

len(qa_data["it_helpdesk"])

15

### Orders & Shipping QA Data
This section contains 15 QA pairs for the Orders & Shipping department.

In [ ]:
# Add 15 QA pairs for Orders & Shipping

qa_data["orders_shipping"] = [
    {
        "question": "How can I track my ShopUNow order?",
        "answer": "You can track your order using the tracking link sent to your email or by entering your order ID on the ShopUNow tracking page."
    },
    {
        "question": "When will my order be shipped?",
        "answer": "Orders ship within 1–2 business days after successful payment confirmation."
    },
    {
        "question": "What shipping carriers does ShopUNow use?",
        "answer": "ShopUNow ships through UPS, FedEx, and USPS depending on location and delivery speed."
    },
    {
        "question": "How do I change my shipping address after placing an order?",
        "answer": "Address changes can be requested within 1 hour of order placement through the 'My Orders' section."
    },
    {
        "question": "Why is my order delayed?",
        "answer": "Delays may occur due to carrier issues, weather, or high-volume periods. Check your tracking link for real-time updates."
    },
    {
        "question": "Can I upgrade my shipping speed after placing the order?",
        "answer": "Yes, if the order has not been shipped. Use the 'Modify Order' option in your account."
    },
    {
        "question": "Do you offer same-day delivery?",
        "answer": "Same-day delivery is available in select cities for eligible items."
    },
    {
        "question": "How do I cancel my order?",
        "answer": "Orders can be canceled within 1 hour of placement from the 'My Orders' dashboard."
    },
    {
        "question": "My tracking number is not working. What should I do?",
        "answer": "Please allow 12–24 hours for the carrier to update tracking information. If it still doesn't update, contact support."
    },
    {
        "question": "What should I do if my order was delivered but I cannot find it?",
        "answer": "Check your mailbox, front desk, neighbors, or building office. If still missing, file a 'Package Not Found' report in your account."
    },
    {
        "question": "Do you ship internationally?",
        "answer": "Yes, international shipping is available to selected countries. Shipping charges vary by destination."
    },
    {
        "question": "How do I request signature delivery?",
        "answer": "Select 'Signature Required' at checkout or modify delivery preferences before shipment."
    },
    {
        "question": "Can I schedule a specific delivery date?",
        "answer": "Scheduled delivery is available for oversized or high-value items. Select a date at checkout."
    },
    {
        "question": "What packaging does ShopUNow use?",
        "answer": "We use eco-friendly recyclable packaging for most items to reduce environmental impact."
    },
    {
        "question": "What do I do if my order arrived damaged?",
        "answer": "Report a damaged delivery within 48 hours through the 'Order Issues' section with photos of the damage."
    }
]

len(qa_data["orders_shipping"])

15

### Returns & Refunds QA Data
This section contains 15 QA pairs for the Returns & Refunds department.

In [ ]:
# Add 15 QA pairs for Returns & Refunds

qa_data["returns_refunds"] = [
    {
        "question": "How do I start a return for my order?",
        "answer": "Go to the 'My Orders' section, select the item you want to return, and click 'Start Return'."
    },
    {
        "question": "What is the return window for ShopUNow purchases?",
        "answer": "Most items can be returned within 30 days of delivery unless specified otherwise."
    },
    {
        "question": "Do I need the original packaging to return an item?",
        "answer": "Original packaging is preferred but not required unless the product description states otherwise."
    },
    {
        "question": "How long does it take to process a refund?",
        "answer": "Refunds are processed within 5–7 business days after the returned item is received and inspected."
    },
    {
        "question": "Can I return an item purchased during a sale?",
        "answer": "Sale items are eligible for return unless marked as 'Final Sale'."
    },
    {
        "question": "Do I have to pay for return shipping?",
        "answer": "Returns are free for defective or incorrect items. For other returns, charges may apply based on your location."
    },
    {
        "question": "What should I do if I received the wrong item?",
        "answer": "Select 'Received Wrong Item' in the return menu and upload a photo if prompted. A replacement will be arranged."
    },
    {
        "question": "Can I exchange an item instead of returning it?",
        "answer": "Yes. Choose the 'Exchange' option during the return process and select the replacement item."
    },
    {
        "question": "How will I know when my refund is issued?",
        "answer": "You will receive an email notification once the refund is processed."
    },
    {
        "question": "Can I return items delivered by third-party sellers?",
        "answer": "Yes, but third-party sellers may have different return policies. Details are available on the product page."
    },
    {
        "question": "What happens if my return is rejected?",
        "answer": "If the returned item does not meet requirements, it will be shipped back to you at no cost."
    },
    {
        "question": "Can I return an opened electronic item?",
        "answer": "Opened electronics are returnable within 15 days unless damaged or missing major components."
    },
    {
        "question": "What if my refund is taking longer than expected?",
        "answer": "Check the status in the 'Refunds' tab. If it shows completed but funds are not received, contact your bank."
    },
    {
        "question": "Can I return an item without a receipt?",
        "answer": "If purchased through your ShopUNow account, no receipt is needed. Guest purchases require the order ID."
    },
    {
        "question": "How do I return a large or heavy item?",
        "answer": "Schedule a pickup using the 'Large Item Return' option. Carrier pickup is usually available within 2–3 business days."
    }
]

len(qa_data["returns_refunds"])

15

### Build Vector Store
This section creates the vector store using all QA data with department metadata.

In [ ]:
!pip install chromadb sentence-transformers

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ------------ --------------------------- 6.8/21.7 MB 41.5 MB/s eta 0:00:01
   ------------------------------- -------- 17.3/21.7 MB 45.7 MB/s eta 0:00:01
   ---------------------------------------- 21.7/21.7 MB 40.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------------------------------------ --- 11.0/12.0 MB 51.6 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 46.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   --------------------------------------- 566.1/566.1 kB 25.5 MB

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

# Create embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize Chroma DB
chroma_client = chromadb.Client()

# Create a collection with metadata support
collection = chroma_client.create_collection(
    name="shopunow_qa",
    metadata={"hnsw:space": "cosine"}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\gayat\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gayat\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Populate Vector Store
This section adds all QA pairs with department metadata into the vector store.

In [ ]:
# Add all QA pairs into the Chroma collection

ids = []
documents = []
metadatas = []

for dept, qa_list in qa_data.items():
    for idx, qa in enumerate(qa_list):
        doc_id = f"{dept}_{idx}"
        ids.append(doc_id)
        documents.append(qa["question"])
        metadatas.append({
            "department": dept,
            "answer": qa["answer"]
        })

# Create embeddings for all questions
embeddings = embedding_model.encode(documents).tolist()

# Add to Chroma collection
collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=embeddings
)

print("Total QA items added to vector store:", len(ids))

Total QA items added to vector store: 60


### Routing and Sentiment Model
This section defines the Pydantic model for sentiment and department.

In [ ]:
from pydantic import BaseModel
from typing import Literal

# Allowed departments (match keys in `departments`)
ALLOWED_DEPARTMENTS = [
    "human_resources",
    "it_helpdesk",
    "orders_shipping",
    "returns_refunds"
]

# Pydantic model for classification result
class RoutingResult(BaseModel):
    sentiment: Literal["positive", "neutral", "negative"]
    department: Literal[
        "human_resources",
        "it_helpdesk",
        "orders_shipping",
        "returns_refunds",
        "unknown"
    ]

### LLM-based Routing and Sentiment
This section uses Groq to classify a query into sentiment and department.=

In [ ]:
import json

def classify_query(query: str) -> RoutingResult:
    system_prompt = """
You classify queries for a retail company called ShopUNow.

Return a JSON object with exactly these two fields:
- "sentiment": one of ["positive", "neutral", "negative"]
- "department": one of ["human_resources", "it_helpdesk", "orders_shipping", "returns_refunds", "unknown"]

Rules:
- If the query is mainly about HR policies, benefits, salary, leave, or employees -> human_resources.
- If the query is about laptops, email, VPN, access, or technical issues -> it_helpdesk.
- If the query is about delivery, tracking, shipping, or packages -> orders_shipping.
- If the query is about returns, refunds, exchanges, or money back -> returns_refunds.
- If none of the above clearly apply, use department = "unknown".

Respond with ONLY valid JSON. No extra text.
"""

    user_prompt = f"Query: {query}"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    content = response.choices[0].message.content
    data = json.loads(content)
    return RoutingResult(**data)

### Test Routing and Sentiment
This section tests the LLM classifier on sample queries.

In [ ]:
# Test the classifier with a few sample queries

test_queries = [
    "My delivery is late. Where is my package?",
    "I cannot log into my company email.",
    "How many vacation days do I get per year?",
    "I want to return a damaged item and get a refund."
]

for q in test_queries:
    result = classify_query(q)
    print("Query:", q)
    print("Sentiment:", result.sentiment)
    print("Department:", result.department)
    print("-" * 40)

Query: My delivery is late. Where is my package?
Sentiment: negative
Department: orders_shipping
----------------------------------------
Query: I cannot log into my company email.
Sentiment: negative
Department: it_helpdesk
----------------------------------------
Query: How many vacation days do I get per year?
Sentiment: neutral
Department: human_resources
----------------------------------------
Query: I want to return a damaged item and get a refund.
Sentiment: negative
Department: returns_refunds
----------------------------------------


### RAG Answer Function
This section uses the vector store and Groq to answer queries for a given department.

In [ ]:
def rag_answer(query: str, department: str) -> str:
    if department not in ALLOWED_DEPARTMENTS:
        return "I cannot find a matching department for this query."

    # Encode the query
    query_embedding = embedding_model.encode([query]).tolist()

    # Retrieve similar questions for this department
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3,
        where={"department": department}
    )

    docs = results.get("documents", [[]])[0]
    metas = results.get("metadatas", [[]])[0]

    # Build a simple context from retrieved answers
    context_parts = []
    for meta in metas:
        context_parts.append(meta["answer"])
    context_text = "\n".join(context_parts)

    system_prompt = """
You are an assistant for ShopUNow.

Use the context below to answer the user's question.
If the answer is not in the context, say you do not have enough information.

Context:
""" + context_text

    user_prompt = f"Question: {query}"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content.strip()

### Handle Query: Routing, RAG, and Human Escalation Form
This section connects classification, RAG, and the human-escalation form into one flow.

In [ ]:
def human_escalation_form(original_query: str) -> str:
    """
    Simple human-escalation form.
    Collects basic user details and shows a confirmation message.
    """
    print("Your query is being escalated to a human support agent.")
    print("Please provide a few details.\n")

    name = input("Enter your name: ")
    email = input("Enter your email: ")
    phone = input("Enter your phone number: ")
    summary = input("Briefly describe your issue: ")

    confirmation = (
        "Thank you. Here is the information collected:\n"
        f"Name: {name}\n"
        f"Email: {email}\n"
        f"Phone: {phone}\n"
        f"Issue summary: {summary}\n"
        f"Original query: {original_query}\n"
        "A human support agent from ShopUNow will contact you soon."
    )

    print("\n" + confirmation)
    return confirmation


def handle_query(user_query: str) -> str:
    """
    Main entry point for the assistant.
    Uses LLM classification, then decides between RAG and human escalation.
    """
    routing = classify_query(user_query)

    print("Routing decision:")
    print("  Sentiment :", routing.sentiment)
    print("  Department:", routing.department)
    print("-" * 40)

    # Human escalation path (stretch goal)
    if routing.sentiment == "negative" or routing.department == "unknown":
        return human_escalation_form(original_query=user_query)

    # RAG answer path
    answer = rag_answer(user_query, routing.department)
    print("AI answer:")
    print(answer)
    return answer

### Test the Full Assistant
This section tests the handle_query function for RAG and human escalation paths.

In [ ]:
# Test 1: Neutral HR query (should use RAG and NOT show the form)
print("=== Test 1: Neutral HR query (RAG path) ===")
handle_query("How many vacation days do I get per year?")
print("\n")

# Test 2: Negative shipping query (should trigger human escalation form)
print("=== Test 2: Negative shipping query (Human escalation path) ===")
handle_query("I am very upset that my order is still not delivered and no one is helping me.")

=== Test 1: Neutral HR query (RAG path) ===
Routing decision:
  Sentiment : neutral
  Department: human_resources
----------------------------------------
AI answer:
I don't have enough information about the vacation policy. You may want to check the HR Portal under 'Company Policies' for more details about the holiday calendar and vacation days.


=== Test 2: Negative shipping query (Human escalation path) ===
Routing decision:
  Sentiment : negative
  Department: orders_shipping
----------------------------------------
Your query is being escalated to a human support agent.
Please provide a few details.



Enter your name:  Gayatri Nair
Enter your email:  gayatrinair203@gmail.com
Enter your phone number:  7134809428
Briefly describe your issue:  website not working



Thank you. Here is the information collected:
Name: Gayatri Nair
Email: gayatrinair203@gmail.com
Phone: 7134809428
Issue summary: website not working
Original query: I am very upset that my order is still not delivered and no one is helping me.
A human support agent from ShopUNow will contact you soon.


'Thank you. Here is the information collected:\nName: Gayatri Nair\nEmail: gayatrinair203@gmail.com\nPhone: 7134809428\nIssue summary: website not working\nOriginal query: I am very upset that my order is still not delivered and no one is helping me.\nA human support agent from ShopUNow will contact you soon.'

In [ ]:
def handle_query_widget(user_query: str) -> str:
    """
    Widget-safe version of the assistant.
    Does NOT call input(), so it works inside ipywidgets callbacks.
    """
    routing = classify_query(user_query)

    print("Routing decision:")
    print("  Sentiment :", routing.sentiment)
    print("  Department:", routing.department)
    print("-" * 40)

    # For negative or unknown, show escalation message (no input())
    if routing.sentiment == "negative" or routing.department == "unknown":
        msg = (
            "Your query is being escalated to a human support agent.\n"
            "Please share your name, email, phone, and issue summary in a follow-up message."
        )
        print(msg)
        return msg

    # RAG answer path
    answer = rag_answer(user_query, routing.department)
    print("AI answer:")
    print(answer)
    return answer

In [ ]:
handle_query_widget("How many vacation days do I get per year?")

Routing decision:
  Sentiment : neutral
  Department: human_resources
----------------------------------------
AI answer:
I don't have enough information about the vacation policy. You may want to check the HR Portal under 'Company Policies' for more details about the holiday calendar and vacation days.


"I don't have enough information about the vacation policy. You may want to check the HR Portal under 'Company Policies' for more details about the holiday calendar and vacation days."

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Textbox for entering a question
question_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Query:',
    layout=widgets.Layout(width='600px')
)

# Button to submit
ask_button = widgets.Button(
    description='Ask'
)

# Output area
response_output = widgets.Output()

def on_ask_button_clicked(b):
    with response_output:
        clear_output()
        query = question_box.value.strip()
        if not query:
            print("Please enter a question.")
            return
        # IMPORTANT: use the widget-safe handler
        handle_query_widget(query)

ask_button.on_click(on_ask_button_clicked)

display(question_box, ask_button, response_output)

Text(value='', description='Query:', layout=Layout(width='600px'), placeholder='Type your question here...')

Button(description='Ask', style=ButtonStyle())

Output()